# Data Wrangling with DataFrames Coding Quiz

Use this Jupyter notebook to find the answers to the quiz in the previous section. There is an answer key in the next part of the lesson.

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType
from pyspark.sql.window import Window
import pyspark.sql.functions as F

# TODOS: 
# 1) import any other libraries you might need
# 2) instantiate a Spark session 
spark = SparkSession.builder.appName("Quiz").getOrCreate()

# 3) read in the data set located at the path "data/sparkify_log_small.json"
path = "data/sparkify_log_small.json"
user_log = spark.read.json(path)

# 4) write code to answer the quiz questions 

In [2]:
user_log.select("*").limit(5).toPandas()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Showaddywaddy,Logged In,Kenneth,M,112,Matthews,232.93342,paid,"Charlotte-Concord-Gastonia, NC-SC",PUT,NextSong,1509380319284,5132,Christmas Tears Will Fall,200,1513720872284,"""Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537....",1046
1,Lily Allen,Logged In,Elizabeth,F,7,Chase,195.23873,free,"Shreveport-Bossier City, LA",PUT,NextSong,1512718541284,5027,Cheryl Tweedy,200,1513720878284,"""Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537....",1000
2,Cobra Starship Featuring Leighton Meester,Logged In,Vera,F,6,Blackwell,196.20526,paid,"Racine, WI",PUT,NextSong,1499855749284,5516,Good Girls Go Bad (Feat.Leighton Meester) (Alb...,200,1513720881284,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",2219
3,Alex Smoke,Logged In,Sophee,F,8,Barker,405.99465,paid,"San Luis Obispo-Paso Robles-Arroyo Grande, CA",PUT,NextSong,1513009647284,2372,Don't See The Point,200,1513720905284,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",2373
4,None,Logged In,Jordyn,F,0,Jones,NaN,free,"Syracuse, NY",GET,Home,1513648531284,1746,None,200,1513720913284,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",1747


In [3]:
user_log.columns

['artist',
 'auth',
 'firstName',
 'gender',
 'itemInSession',
 'lastName',
 'length',
 'level',
 'location',
 'method',
 'page',
 'registration',
 'sessionId',
 'song',
 'status',
 'ts',
 'userAgent',
 'userId']

In [4]:
# Register the DataFrame as a SQL temporary view
user_log.createOrReplaceTempView("user_log_view")

# Question 1

Which page did user id "" (empty string) NOT visit?<br>
<input type='checkbox'> About</input><br>
<input type='checkbox'> Home</input><br>
<input type='checkbox'> Login</input><br>
<input type='checkbox' checked> NextSong</input>

In [5]:
# TODO: write your code to answer question 1
"""
SELECT DISTINCT page
FROM user_log
WHERE userId = ""
"""
user_log.select("page").where(user_log.userId=="").drop_duplicates().show()

+-----+
| page|
+-----+
| Home|
|About|
|Login|
| Help|
+-----+



In [6]:
spark.sql("""
SELECT DISTINCT page
FROM user_log_view
WHERE userId = ""
""").show()

+-----+
| page|
+-----+
| Home|
|About|
|Login|
| Help|
+-----+



# Question 2 - Reflect

What type of user does the empty string user id most likely refer to?

Users who are only visiting the About, Home and Login pages are probably unregistered visitors.

In [7]:
# TODO: use this space to explore the behavior of the user with an empty string
"""
SELECT *
FROM user_log
WHERE userId = ""
"""
user_log.select("*").where(user_log.userId=="").toPandas()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,None,Logged Out,None,None,0,None,NaN,free,None,PUT,Login,NaN,5598,None,307,1513721196284,None,
1,None,Logged Out,None,None,26,None,NaN,paid,None,GET,Home,NaN,428,None,200,1513721274284,None,
2,None,Logged Out,None,None,5,None,NaN,free,None,GET,Home,NaN,2941,None,200,1513722009284,None,
3,None,Logged Out,None,None,5,None,NaN,paid,None,GET,Home,NaN,5243,None,200,1513723183284,None,
4,None,Logged Out,None,None,6,None,NaN,paid,None,PUT,Login,NaN,5243,None,307,1513723184284,None,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331,None,Logged Out,None,None,4,None,NaN,free,None,PUT,Login,NaN,1367,None,307,1513845056284,None,
332,None,Logged Out,None,None,37,None,NaN,paid,None,GET,Home,NaN,5623,None,200,1513845132284,None,
333,None,Logged Out,None,None,26,None,NaN,paid,None,GET,Home,NaN,6890,None,200,1513845761284,None,
334,None,Logged Out,None,None,0,None,NaN,free,None,PUT,Login,NaN,2966,None,307,1513845905284,None,


In [8]:
spark.sql("""
SELECT *
FROM user_log_view
WHERE userId = ""
""").toPandas()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,None,Logged Out,None,None,0,None,NaN,free,None,PUT,Login,NaN,5598,None,307,1513721196284,None,
1,None,Logged Out,None,None,26,None,NaN,paid,None,GET,Home,NaN,428,None,200,1513721274284,None,
2,None,Logged Out,None,None,5,None,NaN,free,None,GET,Home,NaN,2941,None,200,1513722009284,None,
3,None,Logged Out,None,None,5,None,NaN,paid,None,GET,Home,NaN,5243,None,200,1513723183284,None,
4,None,Logged Out,None,None,6,None,NaN,paid,None,PUT,Login,NaN,5243,None,307,1513723184284,None,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331,None,Logged Out,None,None,4,None,NaN,free,None,PUT,Login,NaN,1367,None,307,1513845056284,None,
332,None,Logged Out,None,None,37,None,NaN,paid,None,GET,Home,NaN,5623,None,200,1513845132284,None,
333,None,Logged Out,None,None,26,None,NaN,paid,None,GET,Home,NaN,6890,None,200,1513845761284,None,
334,None,Logged Out,None,None,0,None,NaN,free,None,PUT,Login,NaN,2966,None,307,1513845905284,None,


# Question 3

How many female users do we have in the data set?<br>
<input type='checkbox' checked> 462</input><br>
<input type='checkbox'> 501</input><br>
<input type='checkbox'> 3820</input><br>
<input type='checkbox'> 5844</input>

In [9]:
# TODO: write your code to answer question 3
"""
SELECT COUNT(DISTINCT userId)
FROM user_log
WHERE gender = "F"
"""
user_log.drop_duplicates(subset=["userId"]).where(user_log.gender == "F").count()

462

In [10]:
user_log.where(user_log.gender == "F").select([F.countDistinct("userId").alias("nunique female users")]).show()

+--------------------+
|nunique female users|
+--------------------+
|                 462|
+--------------------+



In [11]:
spark.sql(
"""
SELECT COUNT(DISTINCT userId) AS `nunique female users`
FROM user_log_view
WHERE gender = "F"
"""
).show()

+--------------------+
|nunique female users|
+--------------------+
|                 462|
+--------------------+



# Question 4

How many songs were played from the most played artist?<br>
<input type='checkbox'> 3</input><br>
<input type='checkbox'> 53</input><br>
<input type='checkbox' checked> 83</input><br>
<input type='checkbox'> 113</input>

In [12]:
# TODO: write your code to answer question 4
user_log.filter(user_log.page == 'NextSong') \
    .select('Artist') \
    .groupBy('Artist') \
    .agg(F.count('Artist')) \
    .withColumnRenamed('count(Artist)', 'Artistcount') \
    .sort(F.desc('Artistcount')) \
    .show(1)

+--------+-----------+
|  Artist|Artistcount|
+--------+-----------+
|Coldplay|         83|
+--------+-----------+
only showing top 1 row



In [13]:
spark.sql(
"""
SELECT artist, COUNT(song) AS Artistcount
FROM user_log_view
WHERE page = 'NextSong'
GROUP BY artist
ORDER BY 2 DESC
LIMIT 1
"""
).show()

+--------+-----------+
|  artist|Artistcount|
+--------+-----------+
|Coldplay|         83|
+--------+-----------+



# Question 5 (challenge)

How many songs do users listen to on average between visiting our home page? Please round your answer to the closest integer.<br>
<input type='checkbox'> 5</input><br>
<input type='checkbox' checked> 7</input><br>
<input type='checkbox'> 9</input><br>
<input type='checkbox'> 11</input>

In [14]:
# TODO: write your code to answer question 5
# User defined function
ishome = F.udf(lambda page: int(page == 'Home'), IntegerType())

# Define window function
user_window = Window \
    .partitionBy('userId') \
    .orderBy(F.desc('ts')) \
    .rangeBetween(Window.unboundedPreceding, 0)

In [15]:
# Step 1: Flag page as homevisit; Step 2: Cumulate homevisit as period
cusum = user_log.where((user_log.page == 'NextSong') | (user_log.page == 'Home')) \
            .select(['userId', 'page', 'ts']) \
            .withColumn('homevisit', ishome('page')) \
            .withColumn('period', F.sum('homevisit').over(user_window))

cusum.show()

+------+--------+-------------+---------+------+
|userId|    page|           ts|homevisit|period|
+------+--------+-------------+---------+------+
|  1436|NextSong|1513783259284|        0|     0|
|  1436|NextSong|1513782858284|        0|     0|
|  2088|    Home|1513805972284|        1|     1|
|  2088|NextSong|1513805859284|        0|     1|
|  2088|NextSong|1513805494284|        0|     1|
|  2088|NextSong|1513805065284|        0|     1|
|  2088|NextSong|1513804786284|        0|     1|
|  2088|NextSong|1513804555284|        0|     1|
|  2088|NextSong|1513804196284|        0|     1|
|  2088|NextSong|1513803967284|        0|     1|
|  2088|NextSong|1513803820284|        0|     1|
|  2088|NextSong|1513803651284|        0|     1|
|  2088|NextSong|1513803413284|        0|     1|
|  2088|NextSong|1513803254284|        0|     1|
|  2088|NextSong|1513803057284|        0|     1|
|  2088|NextSong|1513802824284|        0|     1|
|  2162|NextSong|1513781246284|        0|     0|
|  2162|NextSong|151

In [16]:
# Step 3: count period where page = 'NextSong'. We filter page != 'Home' because it is the boundary of each Home and NextSong
# Step 4: Calculate how many songs do users listen to on average between visiting our home page
cusum.where(cusum.page == 'NextSong') \
    .select('*') \
    .groupBy('userId', 'period') \
    .agg({'period': 'count'}) \
    .agg({'count(period)': 'avg'}).show()

+------------------+
|avg(count(period))|
+------------------+
| 6.898347107438017|
+------------------+



In [17]:
# Step 1: Flag page as homevisit
# Step 2: Cumulate homevisit as period
spark.sql("""
WITH homevisit_tb AS
    (
    SELECT 
        userId, page, ts,
        IF(page = 'Home', 1, 0) AS homevisit
    FROM user_log_view
    WHERE page = 'NextSong'
        OR page = 'Home'
    ),
period_tb AS
    (
    SELECT
        *,
        SUM(homevisit) OVER(PARTITION BY userId ORDER BY ts DESC) AS period
    FROM homevisit_tb
    )
SELECT *
FROM period_tb
""").toPandas().head(20)

,userId,page,ts,homevisit,period
0,1436,NextSong,1513783259284,0,0
1,1436,NextSong,1513782858284,0,0
2,2088,Home,1513805972284,1,1
3,2088,NextSong,1513805859284,0,1
4,2088,NextSong,1513805494284,0,1
5,2088,NextSong,1513805065284,0,1
6,2088,NextSong,1513804786284,0,1
7,2088,NextSong,1513804555284,0,1
8,2088,NextSong,1513804196284,0,1
9,2088,NextSong,1513803967284,0,1


In [18]:
# Step 1: Flag page as homevisit
# Step 2: Cumulate homevisit as period
# Step 3: count period where page = 'NextSong'. We filter page != 'Home' because it is the boundary of each Home and NextSong
spark.sql("""
WITH homevisit_tb AS
    (
    SELECT 
        userId, page, ts,
        IF(page = 'Home', 1, 0) AS homevisit
    FROM user_log_view
    WHERE page = 'NextSong'
        OR page = 'Home'
    ),
period_tb AS
    (
    SELECT
        *,
        SUM(homevisit) OVER(PARTITION BY userId ORDER BY ts DESC) AS period
    FROM homevisit_tb
    ),
count_period_tb AS
    (
    SELECT 
        userId, 
        period, 
        COUNT(period) AS count_period
    FROM period_tb
    WHERE page = 'NextSong'
    GROUP BY userId, period
    )
SELECT *
FROM count_period_tb
""").toPandas().head(20)

,userId,period,count_period
0,1436,0,2
1,2088,1,13
2,2162,0,19
3,2162,2,15
4,2294,0,4
5,2294,1,17
6,2294,2,3
7,2294,3,16
8,2294,4,4
9,2294,5,11


In [19]:
# Step 1: Flag page as homevisit
# Step 2: Cumulate homevisit as period
# Step 3: count period where page = 'NextSong'. We filter page != 'Home' because it is the boundary of each Home and NextSong
# Step 4: Calculate how many songs do users listen to on average between visiting our home page
spark.sql("""
WITH homevisit_tb AS
    (
    SELECT 
        userId, page, ts,
        IF(page = 'Home', 1, 0) AS homevisit
    FROM user_log_view
    WHERE page = 'NextSong'
        OR page = 'Home'
    ),
period_tb AS
    (
    SELECT
        *,
        SUM(homevisit) OVER(PARTITION BY userId ORDER BY ts DESC) AS period
    FROM homevisit_tb
    ),
count_period_tb AS
    (
    SELECT 
        userId, 
        period, 
        COUNT(period) AS count_period
    FROM period_tb
    WHERE page = 'NextSong'
    GROUP BY userId, period
    )
SELECT AVG(count_period)
FROM count_period_tb
""").toPandas()

,avg(count_period)
0,6.898347
